In [258]:
from ortools.linear_solver import pywraplp
import numpy as np
from random import randint
import os
import glob

1) **Problema de Transporte:**

In [259]:
def transportationProblem(M,S,D,min=True, isDual = False, completeness=True):
    lenS= len(S)
    lenD= len(D)
    solver = pywraplp.Solver.CreateSolver('GLOP')
    solver.use_dual_simplex = isDual

    if(len(M)==0 or len(M[0])==0):
        raise Exception("Invalid input")
    V=[]
    if(completeness):
        V = [
            [solver.IntVar(0,solver.infinity(),'x{0}{1}'.format(i,j))
                for j in range(lenD)] 
                    for i in range(lenS)
            ]
    else:
        V = [
            [solver.NumVar(0,solver.infinity(),'x{0}{1}'.format(i,j))
                for j in range(lenD)] 
                    for i in range(lenS)
            ]
    
    
    Z = sum(M[i][j]*V[i][j] for i in range(lenS) for j in range(lenD))
    for i in range(lenS):
        solver.Add(
            solver.Sum([V[i][j] for j in range(lenD)]) <= S[i]) 
    for j in range(lenD):
        solver.Add(
            solver.Sum([V[i][j] for i in range(lenS)]) >= D[j])
    
    if(min):
        solver.Minimize(Z)
    else:
        solver.Maximize(Z)
    status = solver.Solve()
    
    if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
        print('Solution:--------------------')
        print('Objective value =', solver.Objective().Value())
        print('\nAdvanced usage:')
        print('\tProblem solved in %f milliseconds' % solver.wall_time())
        print('\tProblem solved in %d iterations' % solver.iterations())
        return True
    else:
        print('<<<<<<<<<<<<<<<<<<<<<<<<<<<<<The problem does not have an optimal solution.>')
        return False

In [260]:
def GenerateData(instances =10, lenS = 100, lenD=100, maxSend=20,  valid=False ):
    #files
    files = glob.glob('data/*')
    for f in files:
        os.remove(f)
    for x in range(instances):
        S = [randint(0,100) for i in range(lenS)]
        D = [randint(0,100) for i in range(lenD)]
        M = [ [randint(0,maxSend) for i in range(lenS)] for j in range(lenD)]

        fo = open("data/dataS{0}.txt".format(x), "w")
        for i in range(lenS): 
            fo.write('{0} '.format(S[x]))
        fo.close()
        fo = open("data/dataD{0}.txt".format(x), "w")
        for i in range(lenD): 
            fo.write('{0} '.format(D[x]))
        fo.close()
        fo = open("data/data{0}.txt".format(x), "w")
        for i in range(lenS): 
            for y in range(lenD): 
                fo.write('{0} '.format(M[i][y]))
            fo.write('\n')
        fo.close()

In [261]:
interations = 10
GenerateData(interations, 100, 100 )       
for i in range(interations):
    M = np.loadtxt('data/data{0}.txt'.format(i))
    S = np.loadtxt('data/dataS{0}.txt'.format(i))
    D = np.loadtxt('data/dataD{0}.txt'.format(i))
    print('#################################################')
    print('Simplex - sem Dual & Sem Integralidade')
    result = transportationProblem(M,S,D,isDual=False,completeness=False)
    if(not result):
        continue
    print('Simplex - sem Dual & com Integralidade')
    transportationProblem(M,S,D,isDual=False,completeness=True)
    print('Simplex - com Dual & Sem Integralidade')
    transportationProblem(M,S,D,isDual=True,completeness=False)
    print('Simplex - com Dual & com Integralidade')
    transportationProblem(M,S,D,isDual=True,completeness=True)

#################################################
Simplex - sem Dual & Sem Integralidade
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<The problem does not have an optimal solution.>
#################################################
Simplex - sem Dual & Sem Integralidade
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<The problem does not have an optimal solution.>
#################################################
Simplex - sem Dual & Sem Integralidade
Solution:--------------------
Objective value = 59.0

Advanced usage:
	Problem solved in 708.000000 milliseconds
	Problem solved in 441 iterations
Simplex - sem Dual & com Integralidade
Solution:--------------------
Objective value = 59.0

Advanced usage:
	Problem solved in 730.000000 milliseconds
	Problem solved in 441 iterations
Simplex - com Dual & Sem Integralidade
Solution:--------------------
Objective value = 59.0

Advanced usage:
	Problem solved in 825.000000 milliseconds
	Problem solved in 441 iterations
Simplex - com Dual & com Integralidade
Solution:------------

2)  **Dual do Problema de Transporte**